# Capacitated Facility Location Problem - Template

In [1]:
from pulp import *
import pandas as pd

In [3]:
# Import farm_cluster_mock_5 dataset - which is the mock data for potential digester locations
potential_digester_location = pd.read_csv(r'./farm_cluster_mock_5.csv')
potential_digester_location.head()

,x,y,count
0,6.554937,52.254895,25
1,6.870686,52.215190,25
2,6.664105,52.392622,32
3,6.730518,52.252885,42
4,6.914234,52.387401,18


In [14]:
# Define the capacities of digester
medium_digester_capacity = 7848  # in tonne/yr
large_digester_capacity = 15056  # in tonne/yr

In [17]:
# Define the costs of digester 
medium_digester_cost = 209249   # in euro CAPEX
large_digester_cost = 252616    # in euro CAPEX

In [ ]:
potential_digester_location = potential_digester_location.drop(['count'], axis=1) # drop unnecessary column

In [19]:
# Create mock digester capacity data
potential_digester_location['capacity'] = [medium_digester_capacity, medium_digester_capacity, large_digester_capacity, large_digester_capacity, large_digester_capacity]

# Create mock digester cost data
potential_digester_location['cost'] = [medium_digester_cost, medium_digester_cost, large_digester_cost, large_digester_cost, large_digester_cost]

In [5]:
farm = pd.read_csv(r"./farm_mock.csv")
farm.head()

,Unnamed: 0,x,y,manure_t,biogas_potential_m3
0,0,6.765843,52.314139,7431,2898090.0
1,1,6.837249,52.425084,7246,2825940.0
2,2,6.859361,52.386225,9639,3759210.0
3,3,6.753483,52.434382,9821,3830190.0
4,4,6.484077,52.243620,2732,1065480.0


In [12]:
# Lists (sets / Array) of Customers and Facilities
Farm = farm.index.tolist()  # Customer = [1,2,3,4,5]
Plant = potential_digester_location.index.tolist()    # Facility = ['Fac-1', 'Fac-2', 'Fac-3']

In [28]:
# Dictionaries of the demands and facilities as well as fixed cost at each facility
Manure_Production = farm['manure_t'].to_dict()  # Demand = {1 : 80, 2 : 270, 3 : 250, 4 : 160, 5 : 180}
Max_Capacity = potential_digester_location['capacity'].to_dict()    # Max_Supply = {'Fac-1' : 500, 'Fac-2' : 500, 'Fac-3' : 500} 
Fixed_Cost = potential_digester_location['cost'].to_dict()  # fixed_cost = {'Fac-1' : 1000, 'Fac-2' : 1000, 'Fac-3' : 1000 }

In [34]:
from scipy.spatial.distance import euclidean

# # Dictionary of the transportation cost from each facility to all the customers 
# transportation_cost = {'Fac-1' : {1 : 4, 2 : 5, 3 : 6, 4 : 8, 5 : 10},
#                        'Fac-2' : {1 : 6, 2 : 4, 3 : 3, 4 : 5, 5 : 8},
#                        'Fac-3' : {1 : 9, 2 : 7, 3 : 4, 4 : 3, 5 : 4}
#                       }

# Create a mock distance matrix to store the euclidean distance between each potential digester location to every farm
transportation_cost = {}

for plant_idx, plant_row in potential_digester_location.iterrows():
    plant_coords = (plant_row['y'], plant_row['x'])
    transportation_cost[plant_idx] = {}  # Create an inner dictionary for each plant

    for farm_idx, farm_row in farm.iterrows():
        farm_coords = (farm_row['y'], farm_row['x'])
        distance = euclidean(plant_coords, farm_coords)
        transportation_cost[plant_idx][farm_idx] = distance

# transportation_cost

In [35]:
# Setting the Problem
prob = LpProblem("Capacitated_Facility_Location_Problem_V1", LpMinimize)

In [41]:
# Defining our Decision Variables
use_plant = LpVariable.dicts("Plant", Plant, LpBinary)    
    # use_facility = LpVariable.dicts("Use Facility", Facility, 0, 1, LpBinary)
ser_farm = LpVariable.dicts("Plant_Farm", [(i,j) for i in Farm for j in Plant], 0)
    # ser_customer = LpVariable.dicts("Service", [(i,j) for i in Customer for j in Facility], 0)

In [43]:
# Setting the Objective Function
prob += lpSum(Fixed_Cost[j]*use_plant[j] for j in Plant) + lpSum(transportation_cost[j][i]*ser_farm[(i,j)] for j in Plant for i in Farm)
    # prob += lpSum(fixed_cost[j]*use_facility[j] for j in Facility) + lpSum(transportation_cost[j][i]*ser_customer[(i,j)] for j in Facility for i in Customer)

In [74]:
# Costraints
for i in Customer:
    prob += lpSum(ser_customer[(i,j)] for j in Facility) == Demand[i]

for j in Facility:
    prob += lpSum(ser_customer[(i,j)] for i in Customer) <= Max_Supply[j]*use_facility[j]

for i in Customer:
    for j in Facility:
        prob += ser_customer[(i,j)] <= Demand[i]*use_facility[j]

In [75]:
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/wenyuc/opt/anaconda3/envs/bioze_env/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/w_/w1t73xzx35j4jtrj94fjskj80000gn/T/4348bf266f1a4ae8a6ae191cbd320ad3-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/w_/w1t73xzx35j4jtrj94fjskj80000gn/T/4348bf266f1a4ae8a6ae191cbd320ad3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 28 COLUMNS
At line 116 RHS
At line 140 BOUNDS
At line 144 ENDATA
Problem MODEL has 23 rows, 18 columns and 63 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 5610 - 0.00 seconds
Cgl0004I processed model has 23 rows, 18 columns (3 integer (3 of which binary)) and 63 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 2.22045e-16
Cbc0038I Solution found of 5610
Cbc0038I Relaxing continuous gives 5610
Cbc0038I Bef

1

In [76]:
status = LpStatus[prob.status]


In [77]:
print("Solution Status = ", LpStatus[prob.status])

Solution Status =  Optimal


In [78]:
# if status == "Optimal":
#     # Retrieve and store the variable values
#     optimal_x = use_facility.varValue
#     optimal_y = ser_customer.varValue

#     # Retrieve and store the optimal objective value
#     optimal_obj_value = prob.objective.value()
# else:
#     optimal_x = None
#     optimal_y = None
#     optimal_obj_value = None

# print("Optimization Status:", status)
# print("Optimal x:", optimal_x)
# print("Optimal y:", optimal_y)
# print("Optimal Objective Value:", optimal_obj_value)

In [79]:
# Print the status of the optimization
print("Optimization Status:", LpStatus[prob.status])

# Print the optimal values of decision variables
for j in Facility:
    print(f"Use Facility {j}: {use_facility[j].varValue}")

for i in Customer:
    for j in Facility:
        print(f"Service customer {i} from Facility {j}: {ser_customer[(i, j)].varValue}")

Optimization Status: Optimal
Use Facility Fac-1: 0.0
Use Facility Fac-2: 1.0
Use Facility Fac-3: 1.0
Service customer 1 from Facility Fac-1: 0.0
Service customer 1 from Facility Fac-2: 80.0
Service customer 1 from Facility Fac-3: 0.0
Service customer 2 from Facility Fac-1: 0.0
Service customer 2 from Facility Fac-2: 270.0
Service customer 2 from Facility Fac-3: 0.0
Service customer 3 from Facility Fac-1: 0.0
Service customer 3 from Facility Fac-2: 150.0
Service customer 3 from Facility Fac-3: 100.0
Service customer 4 from Facility Fac-1: 0.0
Service customer 4 from Facility Fac-2: 0.0
Service customer 4 from Facility Fac-3: 160.0
Service customer 5 from Facility Fac-1: 0.0
Service customer 5 from Facility Fac-2: 0.0
Service customer 5 from Facility Fac-3: 180.0


In [80]:
# Print the solution of Binary Decision Variables
Tolerance = 0.0001
for j in Facility:
    if use_facility[j].varValue > Tolerance:
        print("Estalish Facility at site = ", j)

Estalish Facility at site =  Fac-2
Estalish Facility at site =  Fac-3


In [81]:
# Print the solution of Continuous Decision Variables
for v in prob.variables():
    print(v.name, "=", v.varValue)

Service_(1,_'Fac_1') = 0.0
Service_(1,_'Fac_2') = 80.0
Service_(1,_'Fac_3') = 0.0
Service_(2,_'Fac_1') = 0.0
Service_(2,_'Fac_2') = 270.0
Service_(2,_'Fac_3') = 0.0
Service_(3,_'Fac_1') = 0.0
Service_(3,_'Fac_2') = 150.0
Service_(3,_'Fac_3') = 100.0
Service_(4,_'Fac_1') = 0.0
Service_(4,_'Fac_2') = 0.0
Service_(4,_'Fac_3') = 160.0
Service_(5,_'Fac_1') = 0.0
Service_(5,_'Fac_2') = 0.0
Service_(5,_'Fac_3') = 180.0
Use_Facility_Fac_1 = 0.0
Use_Facility_Fac_2 = 1.0
Use_Facility_Fac_3 = 1.0


In [82]:
# Initialize lists to store assignment information
assigned_farms = {j: [] for j in Facility}

# Collect assigned farms
for i in Facility:
    for j in Customer:
        if ser_customer[(j,i)].varValue > 0.00001:
            assigned_farms[i].append(j)

assigned_farms

{'Fac-1': [], 'Fac-2': [1, 2, 3], 'Fac-3': [3, 4, 5]}

In [83]:
# Print Optimal
print("Total Cost = ", value(prob.objective))

Total Cost =  5610.0
